<a href="https://colab.research.google.com/github/antussa2016/thesis_work/blob/master/model/thesis_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
! git clone https://github.com/antussa2016/thesis_work

fatal: destination path 'thesis_work' already exists and is not an empty directory.


In [30]:
import matplotlib.pyplot as plt
import pandas as pd

In [31]:
df = pd.read_csv('thesis_work/dataset/final_data_for_modeling .csv')

In [32]:
df.head(1)

,twitter_id,stackoverflow_id,question_count,answer_count,reputation,answer_tag,question_tag,Openness,Conscientiousness,Extraversion,Agreeableness,Neuroticism,preferred_tags1,preferred_tags2,preferred_tags3,oop,web-pro,database,mobile
0,10F1,145587,13,500,74297,"['go', 'json', 'http', 'goroutine', 'struct', ...","['go', 'javascript', 'python', 'c++', 'linux',...",0.839299,0.170988,0.083984,0.047367,0.612101,0,json,0,0,1,0,0


In [33]:
df.drop(columns=['twitter_id','stackoverflow_id','question_count','reputation','answer_tag','question_tag','Extraversion'], inplace=True)

In [34]:
df.head(1)

,answer_count,Openness,Conscientiousness,Agreeableness,Neuroticism,preferred_tags1,preferred_tags2,preferred_tags3,oop,web-pro,database,mobile
0,500,0.839299,0.170988,0.047367,0.612101,0,json,0,0,1,0,0


In [ ]:
# Calculate min and max values for the column
min_value = df['answer_count'].min()
max_value = df['answer_count'].max()

# Apply min-max scaling to the column
df['answer_count'] = (df[['answer_count']] - min_value) / (max_value - min_value)

df.head(1)

,answer_count,Openness,Conscientiousness,Agreeableness,Neuroticism,preferred_tags1,preferred_tags2,preferred_tags3,oop,web-pro,database,mobile
0,1.0,0.839299,0.170988,0.047367,0.612101,0,json,0,0,1,0,0


In [35]:
import numpy as np
x= np.array(df.iloc[:,1:5])
y = np.array(df.iloc[:,8:12])


In [8]:
df.iloc[:,1:5]

,Openness,Conscientiousness,Agreeableness,Neuroticism
0,0.839299,0.170988,0.047367,0.612101
1,0.849555,0.100009,0.001654,0.392036
2,0.894954,0.124989,0.009011,0.288220
3,0.689216,0.625448,0.037679,0.223963
4,0.881779,0.042616,0.003885,0.432516
...,...,...,...,...
814,0.888007,0.739341,0.037764,0.342692
815,0.596079,0.105716,0.245279,1.000000
816,0.969066,0.055010,0.028997,0.295512
817,0.917028,0.062395,0.000687,0.461590


In [9]:
df.iloc[:,8:12]

,oop,web-pro,database,mobile
0,0,1,0,0
1,1,0,0,0
2,1,1,0,0
3,1,0,0,0
4,1,1,0,0
...,...,...,...,...
814,0,1,0,0
815,1,1,0,0
816,1,1,0,0
817,1,1,0,0


In [ ]:
y = y.argmax(axis=1)

In [36]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

In [ ]:
y_train.shape


(655,)

In [ ]:
# Initialize and train an SVM classifier with hyperparameter tuning
param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear']}
grid_search = GridSearchCV(SVC(), param_grid, cv=3)
grid_search.fit(X_train, y_train)


best_model = grid_search.best_estimator_


predictions = best_model.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)


Accuracy: 0.6280487804878049


In [ ]:
for i, (pred_label, true_label) in enumerate(zip(predictions, y_test)):
    print(f"Instance {i + 1}:")
    print("Predicted label:", pred_label)
    print("True label:", true_label)
    print("=" * 40)

# **Random forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
rf_classifier = RandomForestClassifier()
param_grid = {
   'n_estimators': [100, 200, 300],
   'max_depth': [None, 10, 20, 30],
   'min_samples_split': [2, 5, 10],
   'min_samples_leaf': [1, 2, 4],
   'max_features': ['auto', 'sqrt', 'log2'],
   'bootstrap': [True, False]
}
random_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_grid, n_iter=100, cv=5, n_jobs=-1, verbose=2, random_state=42)
random_search.fit(X_train, y_train)
best_rf_classifier = random_search.best_estimator_

Fitting 5 folds for each of 100 candidates, totalling 500 fits


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


In [ ]:
from sklearn.metrics import classification_report
best_rf_classifier.fit(X_train, y_train)
y_pred = best_rf_classifier.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
report = classification_report(y_test, y_pred)
# print("report:", report)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Accuracy: 0.6402439024390244


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
for i, (pred_label, true_label) in enumerate(zip(predictions, y_test)):
    print(f"Instance {i + 1}:")
    print("Predicted label:", pred_label)
    print("True label:", true_label)
    print("=" * 40)

In [56]:
X_train.shape

(655, 4)

In [57]:
X_test.shape

(164, 4)

In [58]:
y_train.shape

(655, 4)

In [59]:
y_test.shape

(164, 4)

In [12]:
from tensorflow import keras
from keras.layers import Conv1D, MaxPooling1D, Flatten, LSTM, Dense
from keras.models import Sequential, load_model
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import ast
import json

In [17]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(4, 1)))# add 1D convolutional layer
model.add(MaxPooling1D(pool_size=2))# add max pooling layer
model.add(Flatten())# add flatten layer
model.add(Dense(64, activation='relu'))# add dense layer
model.add(Dense(4, activation='sigmoid'))# add output layer
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # compile the model
model.fit(X_train, y_train, batch_size=32, epochs=10)

Epoch 1/10
21/21 [==============================] - 1s 3ms/step - loss: 0.6536 - accuracy: 0.6198
Epoch 2/10
21/21 [==============================] - 0s 2ms/step - loss: 0.5773 - accuracy: 0.6824
Epoch 3/10
21/21 [==============================] - 0s 2ms/step - loss: 0.5288 - accuracy: 0.6824
Epoch 4/10
21/21 [==============================] - 0s 2ms/step - loss: 0.5216 - accuracy: 0.6824
Epoch 5/10
21/21 [==============================] - 0s 2ms/step - loss: 0.5214 - accuracy: 0.6824
Epoch 6/10
21/21 [==============================] - 0s 2ms/step - loss: 0.5210 - accuracy: 0.6824
Epoch 7/10
21/21 [==============================] - 0s 2ms/step - loss: 0.5208 - accuracy: 0.6824
Epoch 8/10
21/21 [==============================] - 0s 2ms/step - loss: 0.5206 - accuracy: 0.6824
Epoch 9/10
21/21 [==============================] - 0s 2ms/step - loss: 0.5205 - accuracy: 0.6824
Epoch 10/10
21/21 [==============================] - 0s 3ms/step - loss: 0.5197 - accuracy: 0.6824


In [ ]:
test_loss,test_acc=model.evaluate(X_test,y_test)

6/6 [==============================] - 0s 2ms/step - loss: 0.5099 - accuracy: 0.6280


In [20]:
!pip install scikeras

In [37]:
import tensorflow as keras
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, Dense, Dropout,Flatten
import numpy as np
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier
from itertools import product

In [12]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], 1, X_train.shape[1], 1))

In [ ]:
def create_model(filters, kernel_size, pool_size, units):
    model = Sequential([
        Conv1D(filters, kernel_size, activation='relu', input_shape=(4, 1)),
        MaxPooling1D(pool_size),
        Flatten(),
        Dense(units, activation='relu'),
        Dense(4, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Assuming you have your data prepared as X_train, y_train, X_test, y_test
# Make sure X_train and X_test are reshaped to (samples, features, 1)
# X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
# X_test = X_test.reshape((X_test.shape[0], 1, X_train.shape[1], 1))
# Define hyperparameters to search over
filters_options = [32, 64]
kernel_size_options = [3, 5]
pool_size_options = [2, 3]
units_options = [64, 128]

best_accuracy = 0
best_hyperparameters = {}

# Perform manual search
for filters in filters_options:
    for kernel_size in kernel_size_options:
        for pool_size in pool_size_options:
            for units in units_options:
                model = create_model(filters, kernel_size, pool_size, units)
                model.fit(X_train, y_train, epochs=5, verbose=0, batch_size=32)
                _, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
                if test_accuracy > best_accuracy:
                    best_accuracy = test_accuracy
                    best_hyperparameters = {'filters': filters, 'kernel_size': kernel_size,
                                            'pool_size': pool_size, 'units': units}

# Train the best model with the best hyperparameters
best_model = create_model(**best_hyperparameters)
best_model.fit(X_train, y_train, epochs=5, verbose=1, batch_size=32)
test_loss, test_accuracy = best_model.evaluate(X_test, y_test, verbose=0)
print(f"Best Model Test Accuracy: {test_accuracy}")
print(f"Best Hyperparameters: {best_hyperparameters}")

In [ ]:
# Define a function to create the CNN model
def create_model(filters, kernel_size, pool_size, units):
    model = Sequential([
        Conv1D(filters, kernel_size, activation='relu', input_shape=(4, 1)),
        MaxPooling1D(pool_size),
        Flatten(),
        Dense(units, activation='relu'),
        Dense(4, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sigmoid_crossentropy', metrics=['accuracy'])
    return model

# Assuming you have your data prepared as X_train, y_train, X_test, y_test
# Make sure X_train and X_test are reshaped to (samples, features, 1)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Define hyperparameters to search over
filters_options = [32, 64]
kernel_size_options = [3, 5]
pool_size_options = [2, 3]
units_options = [64, 128]

best_accuracy = 0
best_hyperparameters = {}

# Perform manual search
for filters in filters_options:
    for kernel_size in kernel_size_options:
        for pool_size in pool_size_options:
            for units in units_options:
                model = create_model(filters, kernel_size, pool_size, units)
                model.fit(X_train, y_train, epochs=5, verbose=0, batch_size=32)
                _, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
                if test_accuracy > best_accuracy:
                    best_accuracy = test_accuracy
                    best_hyperparameters = {'filters': filters, 'kernel_size': kernel_size,
                                            'pool_size': pool_size, 'units': units}

# Train the best model with the best hyperparameters
best_model = create_model(**best_hyperparameters)
best_model.fit(X_train, y_train, epochs=5, verbose=1, batch_size=32)
test_loss, test_accuracy = best_model.evaluate(X_test, y_test, verbose=0)
print(f"Best Model Test Accuracy: {test_accuracy}")
print(f"Best Hyperparameters: {best_hyperparameters}")

In [66]:
# Make sure X_train and X_test are reshaped to (samples, features, 1)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Define a function to create the CNN model
def create_model():
    model = Sequential([
        Conv1D(64, 3, activation='relu', input_shape=(4, 1)),
        MaxPooling1D(2),
        Flatten(),
        Dense(100, activation='relu'),
        Dense(4, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Create and train the model
model = create_model()
model.fit(X_train, y_train, epochs=5, verbose=1, batch_size=32)

# Evaluate on test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_accuracy}")

Epoch 1/5
21/21 [==============================] - 1s 5ms/step - loss: 1.7557 - accuracy: 0.5252
Epoch 2/5
21/21 [==============================] - 0s 3ms/step - loss: 1.5845 - accuracy: 0.6824
Epoch 3/5
21/21 [==============================] - 0s 3ms/step - loss: 1.6392 - accuracy: 0.6824
Epoch 4/5
21/21 [==============================] - 0s 4ms/step - loss: 1.7686 - accuracy: 0.6824
Epoch 5/5
21/21 [==============================] - 0s 4ms/step - loss: 1.9380 - accuracy: 0.6824
Test Accuracy: 0.6280487775802612


In [ ]:
model = Sequential([
    keras.layers.Dense(100, input_shape=(4,), activation='relu'),
    keras.layers.Dense(4, activation='sigmoid')

])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train,validation_data=(X_test,y_test), epochs=10,batch_size=32)

Epoch 1/10
21/21 [==============================] - 1s 9ms/step - loss: 0.6751 - accuracy: 0.5786 - val_loss: 0.6399 - val_accuracy: 0.6280
Epoch 2/10
21/21 [==============================] - 0s 3ms/step - loss: 0.6132 - accuracy: 0.6824 - val_loss: 0.5868 - val_accuracy: 0.6280
Epoch 3/10
21/21 [==============================] - 0s 2ms/step - loss: 0.5689 - accuracy: 0.6824 - val_loss: 0.5486 - val_accuracy: 0.6280
Epoch 4/10
21/21 [==============================] - 0s 2ms/step - loss: 0.5402 - accuracy: 0.6824 - val_loss: 0.5259 - val_accuracy: 0.6280
Epoch 5/10
21/21 [==============================] - 0s 2ms/step - loss: 0.5266 - accuracy: 0.6824 - val_loss: 0.5141 - val_accuracy: 0.6280
Epoch 6/10
21/21 [==============================] - 0s 3ms/step - loss: 0.5225 - accuracy: 0.6824 - val_loss: 0.5097 - val_accuracy: 0.6280
Epoch 7/10
21/21 [==============================] - 0s 3ms/step - loss: 0.5209 - accuracy: 0.6824 - val_loss: 0.5082 - val_accuracy: 0.6280
Epoch 8/10
21/21 [==

In [ ]:
test_loss,test_acc=model.evaluate(X_test,y_test)

6/6 [==============================] - 0s 4ms/step - loss: 0.5080 - accuracy: 0.6280


In [ ]:
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))

In [ ]:
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
import numpy as np
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Generate or load your data. Make sure it's suitable for LSTM input.

#Define your LSTM model
# model = Sequential([
#     LSTM(64, input_shape=(None, 4), activation='relu'),
#     LSTM(64, activation='relu'),
#     Dense(4, activation='sigmoid')  # Sigmoid activation for multi-label classification
# ])


model = Sequential()
model.add(LSTM(units=64, input_shape=(1, 4)))
model.add(Dense(units=4, activation='sigmoid'))
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50)

# Evaluate the model
accuracy = model.evaluate(X_test, y_test)

# Make predictions on the test set
predictions = model.predict(X_test)

# Threshold the predictions to get binary labels
threshold = 0.5
binary_predictions = (predictions > threshold).astype('int')

# Display the predicted labels and true labels for each instance
for i, (pred_labels, true_labels) in enumerate(zip(binary_predictions, y_test)):
    print(f"Instance {i + 1}:")
    print("Predicted labels:", pred_labels)
    print("True labels:", true_labels)
    print("=" * 40)

Epoch 1/50
21/21 [==============================] - 2s 4ms/step - loss: 0.6819 - accuracy: 0.4458
Epoch 2/50
21/21 [==============================] - 0s 3ms/step - loss: 0.6569 - accuracy: 0.6824
Epoch 3/50
21/21 [==============================] - 0s 3ms/step - loss: 0.6290 - accuracy: 0.6824
Epoch 4/50
21/21 [==============================] - 0s 3ms/step - loss: 0.5976 - accuracy: 0.6824
Epoch 5/50
21/21 [==============================] - 0s 2ms/step - loss: 0.5675 - accuracy: 0.6824
Epoch 6/50
21/21 [==============================] - 0s 3ms/step - loss: 0.5448 - accuracy: 0.6824
Epoch 7/50
21/21 [==============================] - 0s 3ms/step - loss: 0.5318 - accuracy: 0.6824
Epoch 8/50
21/21 [==============================] - 0s 3ms/step - loss: 0.5256 - accuracy: 0.6824
Epoch 9/50
21/21 [==============================] - 0s 3ms/step - loss: 0.5235 - accuracy: 0.6824
Epoch 10/50
21/21 [==============================] - 0s 4ms/step - loss: 0.5225 - accuracy: 0.6824
Epoch 11/50
21/21 [

In [ ]:
test_loss,test_acc=model.evaluate(X_test,y_test)

6/6 [==============================] - 0s 2ms/step - loss: 0.5075 - accuracy: 0.6402


In [ ]:
y.shape

(4, 4)